# 라이브러리

In [1]:
import pandas as pd
import os
import tqdm
import numpy as np
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
raw_fold = 'D:/data/big2/BD1/raw'

# Load

## result 최종-수정(전체)

In [5]:
# about 16.9s
name = '종합_4등급 차량 리스트 양식 result 최종_수정-작업(2023.04.13_PM.04.25.49)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 36 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   차대번호        1502235 non-null  object 
 1   차량번호        1502235 non-null  object 
 2   제원관리번호      1502235 non-null  object 
 3   차종          1502235 non-null  object 
 4   용도          1502235 non-null  object 
 5   최초등록일       1502235 non-null  int64  
 6   연식          1502235 non-null  int64  
 7   제작일자        1335178 non-null  float64
 8   검사유효일       1502223 non-null  float64
 9   배출가스인증번호    1382222 non-null  object 
 10  등급          1502235 non-null  int64  
 11  차명          1502235 non-null  object 
 12  차종분류        1502235 non-null  object 
 13  차종유형        1502235 non-null  object 
 14  자동차형식       1501958 non-null  object 
 15  제작사명        1492590 non-null  object 
 16  연료          1502235 non-null  object 
 17  엔진형식        1500400 non-null  object 
 18  총중량         1502234 no

In [7]:
df['배인번호_수정'] = df['배인번호_수정'].astype('str')

In [8]:
df.loc[df['배인번호_수정'].str.contains('HB')].shape

(2672, 36)

In [9]:
df.loc[df['배인번호_수정'].str.contains('HB'), '등급_수정'].value_counts(dropna=False)

4.00    2672
Name: 등급_수정, dtype: int64

In [10]:
df.loc[df['배인번호_수정'].str.contains('HB'), '비고'].value_counts(dropna=False)

푸조 307모델 DPF 장착 확인          1677
NaN                          868
푸조407 607모델 혼제 DPF 장착 확인     127
Name: 비고, dtype: int64

## 한불모터스 회신파일

In [25]:
name = '한불모터스_배출가스인증번호_검토자료_회신(2023.04.17)'
file_name = f'{name}.xlsx'
hb = pd.read_excel(os.path.join(raw_fold, file_name))

In [26]:
hb.shape

(2672, 31)

# 확인

In [27]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고'],
      dtype='object')

In [28]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'EM Std.', 'DPF 유무', '등급.1'],
      dtype='object')

In [29]:
hb['EM Std.'].unique()

array(['EURO 4 ENGINE EMISSION CONTROL',
       'EURO 3 L4 VEHICLE TECHNICAL EMISSION CONTROL',
       'EURO 5 L6 UPPER ENGINE EMISSION CONTROl'], dtype=object)

# 전처리

In [30]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             323019
LPG(액화석유가스)      25400
알코올                  1
기타연료                 1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

## 한불모터스 배인번호_수정

In [31]:
df[['배출가스인증번호', '배인번호_수정']].head()

,배출가스인증번호,배인번호_수정
0,4MY-AD-20,4MY-AD-20
1,4MY-AD-20,4MY-AD-20
2,4MY-AD-20,4MY-AD-20
3,4MY-AD-20,4MY-AD-20
4,4MY-AD-20,4MY-AD-20


In [32]:
hb['배인번호_수정'] = hb['배출가스인증번호']

In [33]:
hb['배인번호_수정'].isnull().sum()

0

## 한불모터스 등급_수정

In [34]:
hb['등급.1'].value_counts(dropna=False)

NaN     2671
3.00       1
Name: 등급.1, dtype: int64

In [35]:
hb.loc[hb['등급.1'] == 3, '등급_수정'] = 3

In [36]:
hb['등급_수정'].value_counts(dropna=False)

NaN     2671
3.00       1
Name: 등급_수정, dtype: int64

In [37]:
hb.loc[hb['등급_수정'].isnull() == True, '등급_수정'] = 4

In [38]:
hb['등급_수정'].value_counts(dropna=False)

4.00    2671
3.00       1
Name: 등급_수정, dtype: int64

## 한불모터스 DPF유무_수정

In [39]:
df['DPF유무_수정'].unique()

array(['무', '유', '확인불가', nan], dtype=object)

In [40]:
# WITHOUT PARTICLE FILTER: DPF 없음
hb['DPF 유무'].value_counts(dropna=False)

WITH PARTICLE EMISSION FILTER    2666
WITHOUT PARTICLE FILTER             6
Name: DPF 유무, dtype: int64

In [41]:
hb['DPF 유무'].unique()

array(['WITH PARTICLE EMISSION FILTER', 'WITHOUT PARTICLE FILTER'],
      dtype=object)

In [42]:
hb.loc[hb['DPF 유무'] == 'WITH PARTICLE EMISSION FILTER', 'DPF유무_수정'] = '유'
hb.loc[hb['DPF 유무'] == 'WITHOUT PARTICLE FILTER', 'DPF유무_수정'] = '무'

In [43]:
hb['DPF유무_수정'].value_counts(dropna=False)

유    2666
무       6
Name: DPF유무_수정, dtype: int64

## df에서 hb으로 비고 복제

In [44]:
df.loc[df['배인번호_수정'].str.contains('HB'), '비고'].shape

(2672,)

In [45]:
hb_etc = df.loc[df['배인번호_수정'].str.contains('HB'), ['차대번호', '비고']].reset_index(drop=True)
hb_etc.shape

(2672, 2)

In [46]:
hb = hb.merge(hb_etc, on='차대번호', how='left')
hb.shape

(2672, 35)

In [48]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'EM Std.', 'DPF 유무', '등급.1', '배인번호_수정', '등급_수정', 'DPF유무_수정',
       '비고'],
      dtype='object')

## 한불모터스 컬럼명 정리

In [49]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고'],
      dtype='object')

In [50]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'EM Std.', 'DPF 유무', '등급.1', '배인번호_수정', '등급_수정', 'DPF유무_수정',
       '비고'],
      dtype='object')

In [51]:
hb = hb.rename(columns={'EM Std.':'인증번호 수정 유무'})
hb['인증번호 수정 유무'] = np.nan

In [52]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', 'DPF 유무', '등급.1', '배인번호_수정', '등급_수정', 'DPF유무_수정',
       '비고'],
      dtype='object')

In [53]:
hb.drop(['DPF 유무', '등급.1'], axis=1, inplace=True)

In [54]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '비고'],
      dtype='object')

## 한불모터스 샘플 result와 부분 교체

### df에서 한불모터스 샘플 제거
- 배인번호_수정에 'HB' 포함된 샘플

In [55]:
df.loc[df['배인번호_수정'].str.contains('HB')].shape

(2672, 36)

In [56]:
pop_idx = df.loc[df['배인번호_수정'].str.contains('HB')].index
len(pop_idx)

2672

In [58]:
min(pop_idx), max(pop_idx)

(3593, 999600)

In [57]:
df.shape

(1502235, 36)

In [60]:
df_pop_hb = df.drop(pop_idx).reset_index(drop=True)
df_pop_hb.shape

(1499563, 36)

### 한불모터스 제거한 df와 한불모터스 회신 파일 병합

In [61]:
total = pd.concat([df_pop_hb, hb], ignore_index=True)
total.shape

(1502235, 36)

In [62]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고'],
      dtype='object')

In [64]:
df['비고'].isnull().sum()

1455741

In [65]:
df['비고'].unique()

array([nan, '현재 5등급으로 반영되어 있음(2/23)',
       '등급변경 5등급 확인 4MY-DB-04 인증번호 5등급 차량, DPF 확인불가->무 로 변경',
       'BX210, BX120 혼재 및 5등급(BX212)에 동일 차종 있어 등급 검토, 전체 06, 07년식 차량',
       '말소', 'DPF 장착 확인 필요', '4MY-DB-03 동일차량 5등급',
       '동일 인증번호 3등급에도 있음. 확인 필요',
       'DPF 무, 동일차종 5등급 차량 및 저감사업으로 DPF장착지원 되고 있는 차량 J3엔진 123마력 2902cc 형식 WFK12F8',
       '푸조 307모델 DPF 장착 확인', '푸조407 607모델 혼제 DPF 장착 확인', 'DPF장착 확인 필요',
       'DPF 장착 확인(미장착) 검토 필요',
       '동일인정번호 차량 중 17대 보조금 지급으로 DPF장착. 실제 출고시 DPF장착 유무 확인 필요',
       '동일인증번호 차량 중 배출가스 저감창치 장착된 차량 있음. 확인 필요', 'DPF 장착 확인(무, 확인불가)',
       'PDF 확인불가', 'TDV6 DPF장착 확인 무, 확인불가',
       '짚 그랜드 체로키 8MY-DK-14-01 배출가스 인증번호 현재 3등급으로 분류 중',
       '짚 그랜드 체로키 동일 제원관리번호 차량 4,5등급 혼재, 8MY-DK-14-01 배출가스 인증번호 3등급 분류중',
       '짚 그랜드 체로키 동일 제원관리번호 차량 5등급, 8MY-DK-14-01 배출가스 인증번호 3등급 분류중',
       '짚 그랜드 체로키 동일 배출가스인증번로, 동일제원관리번호 3등급으로 분류 되어있음.',
       '제원관리번호 확인 04220000400031308', '제원관리번호 확인 04220000400041309',
       '제원관리번호 확인 04220000800021308', '제원관리번호 확인 042

In [63]:
total['비고'].isnull().sum()

1455741

In [66]:
total['비고'].unique()

array([nan, '현재 5등급으로 반영되어 있음(2/23)',
       '등급변경 5등급 확인 4MY-DB-04 인증번호 5등급 차량, DPF 확인불가->무 로 변경',
       'BX210, BX120 혼재 및 5등급(BX212)에 동일 차종 있어 등급 검토, 전체 06, 07년식 차량',
       '말소', 'DPF 장착 확인 필요', '4MY-DB-03 동일차량 5등급',
       '동일 인증번호 3등급에도 있음. 확인 필요',
       'DPF 무, 동일차종 5등급 차량 및 저감사업으로 DPF장착지원 되고 있는 차량 J3엔진 123마력 2902cc 형식 WFK12F8',
       'DPF장착 확인 필요', 'DPF 장착 확인(미장착) 검토 필요',
       '동일인정번호 차량 중 17대 보조금 지급으로 DPF장착. 실제 출고시 DPF장착 유무 확인 필요',
       '동일인증번호 차량 중 배출가스 저감창치 장착된 차량 있음. 확인 필요', 'DPF 장착 확인(무, 확인불가)',
       'PDF 확인불가', 'TDV6 DPF장착 확인 무, 확인불가',
       '짚 그랜드 체로키 8MY-DK-14-01 배출가스 인증번호 현재 3등급으로 분류 중',
       '짚 그랜드 체로키 동일 제원관리번호 차량 4,5등급 혼재, 8MY-DK-14-01 배출가스 인증번호 3등급 분류중',
       '짚 그랜드 체로키 동일 제원관리번호 차량 5등급, 8MY-DK-14-01 배출가스 인증번호 3등급 분류중',
       '짚 그랜드 체로키 동일 배출가스인증번로, 동일제원관리번호 3등급으로 분류 되어있음.',
       '제원관리번호 확인 04220000400031308', '제원관리번호 확인 04220000400041309',
       '제원관리번호 확인 04220000800021308', '제원관리번호 확인 04220000800031308',
       '봉고3 8MY-KM-13-19 동일 배출가스 인증번호 3,

## 등급_수정

In [67]:
total['등급_수정'].value_counts(dropna=False)

4.00    1149290
NaN      348422
3.00       4397
5.00        126
Name: 등급_수정, dtype: int64

In [68]:
total['연료'].value_counts(dropna=False)

경유             1153813
휘발유             323019
LPG(액화석유가스)      25400
알코올                  1
기타연료                 1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [69]:
total['등급_수정'] = total['등급_수정'].fillna(4)

In [70]:
total['등급_수정'].value_counts(dropna=False)

4.00    1497712
3.00       4397
5.00        126
Name: 등급_수정, dtype: int64

#### 컬럼 정리

In [72]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고'],
      dtype='object')

#### 출력

In [73]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.04.18_PM.01.15.59'

In [74]:
# about 21.8s
name = f'4등급_차량_리스트_양식_result_최종(전체)({today_date})'
file_name = f'{name}.csv'
total.to_csv(os.path.join(raw_fold, file_name), index=False, encoding='cp949')

In [75]:
name = f'4등급_차량_리스트_양식_result_최종(전체)({today_date})'
name

'4등급_차량_리스트_양식_result_최종(전체)(2023.04.18_PM.01.15.59)'

In [76]:
# about 14m 10.5s
total.iloc[:700000].to_excel(os.path.join(raw_fold, f'4등급_차량_리스트_양식_result_최종(전체)({today_date})01.xlsx'), index=False)
total.iloc[700000:].to_excel(os.path.join(raw_fold, f'4등급_차량_리스트_양식_result_최종(전체)({today_date})02.xlsx'), index=False)

# 코드 마지막